In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier


In [2]:

def score_classifier(dataset,classifier,labels):

    """
    performs 3 random trainings/tests to build a confusion matrix and prints results with precision and recall scores
    :param dataset: the dataset to work on
    :param classifier: the classifier to use
    :param labels: the labels used for training and validation
    :return:
    """

    kf = KFold(n_splits=3,random_state=50,shuffle=True)
    confusion_mat = np.zeros((2,2))
    # Recall = TP / (TP + FN)
    
    recall = 0
    for training_ids,test_ids in kf.split(dataset):
        training_set = dataset[training_ids]
        training_labels = labels[training_ids]
        test_set = dataset[test_ids]
        test_labels = labels[test_ids]
        classifier.fit(training_set,training_labels)
        predicted_labels = classifier.predict(test_set)
        confusion_mat+=confusion_matrix(test_labels,predicted_labels)
        recall += recall_score(test_labels, predicted_labels)
    recall/=3
    print(confusion_mat)
    return recall 



In [3]:

# Load dataset
df = pd.read_csv("nba_logreg.csv")

# extract names, labels, features names and values
names = df['Name'].values.tolist() # players names
labels = df['TARGET_5Yrs'].values # labels
paramset = df.drop(['TARGET_5Yrs','Name'],axis=1).columns.values
df_vals = df.drop(['TARGET_5Yrs','Name'],axis=1).values

# replacing Nan values (only present when no 3 points attempts have been performed by a player)
for x in np.argwhere(np.isnan(df_vals)):
    df_vals[x]=0.0

# normalize dataset
X = MinMaxScaler().fit_transform(df_vals)

#example of scoring with support vector classifier
score_classifier(X,SVC(),labels)


[[271. 238.]
 [145. 686.]]


0.82551959002102

In [2]:

# define a list of classifiers to try
classifiers = [SVC(), RandomForestClassifier(), LogisticRegression(), KNeighborsClassifier(),
 RidgeClassifier(), AdaBoostClassifier(), GradientBoostingClassifier(), MLPClassifier(), LinearDiscriminantAnalysis(),
 QuadraticDiscriminantAnalysis(), KNeighborsClassifier()]

# train each classifier and evaluate performance using score_classifier function
best_recall = 0
best_clf = None
for clf in classifiers:
    score = score_classifier(X, clf, labels)
    if score > best_recall:
        best_recall = score
        best_clf = clf

print("Best classifier with recall score of {}: {}".format(best_recall, best_clf))
import pickle

# best_clf is the trained model
with open('model.pkl', 'wb') as f:
    pickle.dump(best_clf, f)




NameError: name 'SVC' is not defined

In [2]:
from flask import Flask, jsonify, request
import pickle
import numpy as np

app = Flask(__name__)

# Load the trained model

with open('model.pkl', 'rb') as f:
    model = pickle.load(f)


# Define the API endpoint
@app.route('/predict', methods=['POST'])
def predict():
    # Get the request data as a dictionary
    data = request.get_json(force=True)

    # Transform the data into a numpy array
    data_arr = np.array([data['GP'], data['MIN'], data['PTS'], data['FGM'], data['FGA'], data['FG%'], data['3P Made'], data['3PA'], data['3P%'], data['FTM'], data['FTA'], data['FT%'], data['OREB'], data['DREB'], data['REB'], data['AST'], data['STL'], data['BLK'], data['TOV']]).reshape(1, -1)

    # Make a prediction
    prediction = model.predict(data_arr)

    # Convert the prediction to a boolean value
    prediction = bool(prediction[0])

    # Return the prediction as a JSON response
    return jsonify(prediction=prediction)

if __name__ == "__main__":
     app.run(debug=True ,port=8080,use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [21/Apr/2023 15:47:53] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2023 15:47:54] "GET /favicon.ico HTTP/1.1" 404 -
